In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

electives are from the same list as the general mecheng program

In [2]:
url = "http://www.calendar.ubc.ca/vancouver/index.cfm?tree=12,195,272,43"

In [3]:
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

# Aerospace

## 1. Collect course codes

In [4]:
table = page_soup.find("p", text = "Mechanical Engineering Aerospace Option").findNext("table")
containers = table.findAll("td")
len(containers)

54

In [5]:
course_codes = [container.text.strip() for container in containers if bool(re.match("[A-Z]{4} [0-9]{3}", container.text.strip()))]
course_codes

['ELEC 344',
 'MECH 305',
 'MECH 327',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 375',
 'MECH 380',
 'MECH 426',
 'MECH 466',
 'MECH 481',
 'APSC 450',
 'MECH 431',
 'MECH 453 or APSC 4962',
 'MECH 462',
 'MECH 463',
 'MECH 477',
 'MECH 479',
 'MECH 484',
 'MECH 485',
 'MECH 489',
 'MTRL 494']

In [6]:
len(course_codes)

22

In [7]:
course_codes = [course_code.split(" or ") for course_code in course_codes]
course_codes = [course_code for sub_list in course_codes for course_code in sub_list]
course_codes

['ELEC 344',
 'MECH 305',
 'MECH 327',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 375',
 'MECH 380',
 'MECH 426',
 'MECH 466',
 'MECH 481',
 'APSC 450',
 'MECH 431',
 'MECH 453',
 'APSC 4962',
 'MECH 462',
 'MECH 463',
 'MECH 477',
 'MECH 479',
 'MECH 484',
 'MECH 485',
 'MECH 489',
 'MTRL 494']

In [8]:
course_codes = [course_code[:-1] if len(course_code) > 8 else course_code for course_code in course_codes]
course_codes

['ELEC 344',
 'MECH 305',
 'MECH 327',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 375',
 'MECH 380',
 'MECH 426',
 'MECH 466',
 'MECH 481',
 'APSC 450',
 'MECH 431',
 'MECH 453',
 'APSC 496',
 'MECH 462',
 'MECH 463',
 'MECH 477',
 'MECH 479',
 'MECH 484',
 'MECH 485',
 'MECH 489',
 'MTRL 494']

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [9]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "MTRL": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MTRL",
    "ELEC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=ELEC",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC"
}

In [10]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  ELEC 344
scraped  MECH 305
scraped  MECH 327
scraped  MECH 328
scraped  MECH 358
scraped  MECH 360
scraped  MECH 375
scraped  MECH 380
scraped  MECH 426
scraped  MECH 466
scraped  MECH 481
scraped  APSC 450
scraped  MECH 431
scraped  MECH 453
scraped  APSC 496
scraped  MECH 462
scraped  MECH 463
scraped  MECH 477
scraped  MECH 479
scraped  MECH 484
scraped  MECH 485
scraped  MECH 489
scraped  MTRL 494
23


## 3. Inspect and write to CSV

In [11]:
course_names

['Applied Electronics and Electromechanics',
 'Data Analysis and Mechanical Engineering Laboratories',
 'Thermodynamics II',
 'Mechanical Engineering Design Project',
 'Engineering Analysis',
 'Mechanics of Materials',
 'Heat Transfer',
 'Fluid Dynamics',
 'Mechanical Design',
 'Automatic Control',
 'Aerodynamics of Aircraft I',
 'Professional Engineering Practice',
 'Engineering Economics',
 'Aerospace Design Project',
 'Interdisciplinary Engineering Design Project',
 'Finite Element Analysis',
 'Mechanical Vibrations',
 'Aerospace Propulsion',
 'Introduction to Computational Fluid Dynamics',
 'Aircraft Design: Aerodynamics',
 'Aircraft Design: Structures',
 'Experimental Thermofluids',
 'Composite Materials']

In [12]:
course_descs

['Characteristics of semiconductor devices; analog circuits; force and torque production; motor principles and torque-speed characteristics; principles of power electronics. Not open to students in Electrical and Computer Engineering. This course is not eligible for Credit/D/Fail grading. [2-2*-2*] Prerequisite: One of ELEC 201, ELEC 203, MECH 221.',
 'The use of probability and statistical methods for engineering applications. Mechanics of materials, heat transfer, and fluid dynamics. Instrumentation, data acquisition, and data manipulation using modern computational tools. This course is not eligible for Credit/D/Fail grading. [3-5-1*] Prerequisite: All of MECH 220, MECH 223, MECH 224, MECH 225.',
 'Air standard and vapor cycles; first and second law of cycles. Exergy. Gas mixtures.  Energy conservation. Equilibrium. Reacting systems. Application to thermofluid systems such as power plants. This course is not eligible for Credit/D/Fail grading. [3-0-1] Prerequisite: Either (a) MECH 2

In [13]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,ELEC 344,Applied Electronics and Electromechanics,Characteristics of semiconductor devices; anal...
1,MECH 305,Data Analysis and Mechanical Engineering Labor...,The use of probability and statistical methods...
2,MECH 327,Thermodynamics II,Air standard and vapor cycles; first and secon...
3,MECH 328,Mechanical Engineering Design Project,"Applying the design process, making educated a..."
4,MECH 358,Engineering Analysis,Fourier series; auto- and cross-correlation; p...
5,MECH 360,Mechanics of Materials,"Beam deflections, singularity functions; use o..."
6,MECH 375,Heat Transfer,Steady and transient conduction. Radiation hea...
7,MECH 380,Fluid Dynamics,"Review of principles, compressible flow, poten..."
8,MECH 426,Mechanical Design,Design of shafts and welded joints. Fatigue an...
9,MECH 466,Automatic Control,Process and system characteristics; transient ...


In [14]:
df.to_csv('UBC_MechEng_Aerospace_option_core_(Years2-4)_Courses.csv', index = False)

# Biomedical

## 1. Collect course codes

In [15]:
url = "http://www.calendar.ubc.ca/vancouver/index.cfm?tree=12,195,272,43"

In [16]:
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [17]:
#only scrape the first table, the second table in this section is missing mech 326 and otherwise equivalent
table = page_soup.find("h4", text = "Biomechanics and Medical Devices Option").findNext("table")
containers = table.findAll("td")
len(containers)

50

In [18]:
course_codes = [container.text.strip() for container in containers if bool(re.match("[A-Z]{4} [0-9]{3}", container.text.strip()))]
course_codes

['BMEG 410',
 'MECH 305',
 'MECH 325',
 'MECH 326 or MECH 426',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 368',
 'MECH 375',
 'MECH 380',
 'MECH 463',
 'APSC 450',
 'BMEG 456',
 'ELEC 344',
 'MECH 431',
 'MECH 439',
 'MECH 4591',
 'MECH 466',
 'MTRL 495']

In [19]:
len(course_codes)

19

In [20]:
course_codes = [course_code.split(" or ") for course_code in course_codes]
course_codes = [course_code for sub_list in course_codes for course_code in sub_list]
course_codes = [course_code[:-1] if len(course_code) > 8 else course_code for course_code in course_codes]
course_codes

['BMEG 410',
 'MECH 305',
 'MECH 325',
 'MECH 326',
 'MECH 426',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 368',
 'MECH 375',
 'MECH 380',
 'MECH 463',
 'APSC 450',
 'BMEG 456',
 'ELEC 344',
 'MECH 431',
 'MECH 439',
 'MECH 459',
 'MECH 466',
 'MTRL 495']

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [21]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "MTRL": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MTRL",
    "ELEC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=ELEC",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC",
    "BMEG": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=BMEG"
}

In [22]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  BMEG 410
scraped  MECH 305
scraped  MECH 325
Cannot find  MECH 326
scraped  MECH 426
scraped  MECH 328
scraped  MECH 358
scraped  MECH 360
scraped  MECH 368
scraped  MECH 375
scraped  MECH 380
scraped  MECH 463
scraped  APSC 450
scraped  BMEG 456
scraped  ELEC 344
scraped  MECH 431
scraped  MECH 439
scraped  MECH 459
scraped  MECH 466
scraped  MTRL 495
19


In [23]:
course_codes = [code for code in course_codes if code != "MECH 326"] #326 does not exist on website
len(course_codes)

19

## 3. Inspect and write to CSV

In [24]:
course_names

['Biomedical Equipment, Physiology, and Anatomy',
 'Data Analysis and Mechanical Engineering Laboratories',
 'Machine Design',
 'Mechanical Design',
 'Mechanical Engineering Design Project',
 'Engineering Analysis',
 'Mechanics of Materials',
 'Engineering Measurements and Instrumentation',
 'Heat Transfer',
 'Fluid Dynamics',
 'Mechanical Vibrations',
 'Professional Engineering Practice',
 'Clinical and Industrial Biomedical Engineering',
 'Applied Electronics and Electromechanics',
 'Engineering Economics',
 'Biomechanics Research',
 'Biomedical Design Project',
 'Automatic Control',
 'Biomaterials']

In [25]:
course_descs

['Principles and operation of biomedical equipment for cardiovascular system, respiratory system, renal dialysis, endoscopy, surgery, and imaging. Functional relationships of biomedical equipment to physiology and anatomy of major body systems. Credit will be granted for only one of BMEG 410 or BMEG 510. [3-0-2*]',
 'The use of probability and statistical methods for engineering applications. Mechanics of materials, heat transfer, and fluid dynamics. Instrumentation, data acquisition, and data manipulation using modern computational tools. This course is not eligible for Credit/D/Fail grading. [3-5-1*] Prerequisite: All of MECH 220, MECH 223, MECH 224, MECH 225.',
 'Selection of flexible drives, bearings, fluid power system components, and couplings. Design of bolted joints and power screws. Design and selection of gears, gear trains, and mechanisms. This course is not eligible for Credit/D/Fail grading. [3-0-3] Prerequisite: One of MECH 223, MECH 260. Corequisite: MECH 360.',
 'Design

In [26]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,BMEG 410,"Biomedical Equipment, Physiology, and Anatomy",Principles and operation of biomedical equipme...
1,MECH 305,Data Analysis and Mechanical Engineering Labor...,The use of probability and statistical methods...
2,MECH 325,Machine Design,"Selection of flexible drives, bearings, fluid ..."
3,MECH 426,Mechanical Design,Design of shafts and welded joints. Fatigue an...
4,MECH 328,Mechanical Engineering Design Project,"Applying the design process, making educated a..."
5,MECH 358,Engineering Analysis,Fourier series; auto- and cross-correlation; p...
6,MECH 360,Mechanics of Materials,"Beam deflections, singularity functions; use o..."
7,MECH 368,Engineering Measurements and Instrumentation,Industrial measurement needs including: archit...
8,MECH 375,Heat Transfer,Steady and transient conduction. Radiation hea...
9,MECH 380,Fluid Dynamics,"Review of principles, compressible flow, poten..."


In [27]:
df.to_csv('UBC_MechEng_Biomedical_option_core_(Years2-4)_Courses.csv', index = False)

# Mechatronics

## 1. Collect course codes

In [29]:
url = "http://www.calendar.ubc.ca/vancouver/index.cfm?tree=12,195,272,43"
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [30]:
table = page_soup.find("h4", text = "Mechatronics Option").findNext("table")
containers = table.findAll("td")
course_codes = [container.text.strip() for container in containers if bool(re.match("[A-Z]{4} [0-9]{3}", container.text.strip()))]
course_codes

['CPEN 312',
 'CPSC 259',
 'ELEC 302',
 'ELEC 343',
 'MECH 306',
 'MECH 325',
 'MECH 328',
 'MECH 360',
 'MECH 366',
 'MECH 375',
 'MECH 392',
 'MECH 463',
 'APSC 450',
 'CPEN 333',
 'MECH 420',
 'MECH 421',
 'MECH 423',
 'MECH 431',
 'MECH 4581',
 'MECH 467']

In [31]:
course_codes = [course_code[:-1] if len(course_code) > 8 else course_code for course_code in course_codes]
course_codes

['CPEN 312',
 'CPSC 259',
 'ELEC 302',
 'ELEC 343',
 'MECH 306',
 'MECH 325',
 'MECH 328',
 'MECH 360',
 'MECH 366',
 'MECH 375',
 'MECH 392',
 'MECH 463',
 'APSC 450',
 'CPEN 333',
 'MECH 420',
 'MECH 421',
 'MECH 423',
 'MECH 431',
 'MECH 458',
 'MECH 467']

In [32]:
len(course_codes)

20

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [33]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "MATH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=MATH",
    "ELEC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=ELEC",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC",
    "CPEN": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=CPEN",
    "CPSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=CPSC"
}

In [34]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  CPEN 312
scraped  CPSC 259
scraped  ELEC 302
scraped  ELEC 343
scraped  MECH 306
scraped  MECH 325
scraped  MECH 328
scraped  MECH 360
scraped  MECH 366
scraped  MECH 375
scraped  MECH 392
scraped  MECH 463
scraped  APSC 450
scraped  CPEN 333
scraped  MECH 420
scraped  MECH 421
scraped  MECH 423
scraped  MECH 431
scraped  MECH 458
scraped  MECH 467
20


## 3. Inspect and write to CSV

In [36]:
course_names

['Digital Systems and Microcomputers',
 'Data Structures and Algorithms for Electrical Engineers',
 'Electronic Circuits for Electromechanical Design',
 'Electromechanics',
 'Data Analysis and Mechatronics Laboratories',
 'Machine Design',
 'Mechanical Engineering Design Project',
 'Mechanics of Materials',
 'Modelling of Mechatronic Systems',
 'Heat Transfer',
 'Manufacturing Processes',
 'Mechanical Vibrations',
 'Professional Engineering Practice',
 'System Software Engineering',
 'Sensors and Actuators',
 'Mechatronics System Instrumentation',
 'Mechatronic Product Design',
 'Engineering Economics',
 'Mechatronics Design Project',
 'Computer Control of Mechatronics Systems']

In [37]:
course_descs

['Data representation in digital computers; boolean algebra; the design and optimization and implementation of combinatorial and sequential circuits; modern digital circuit technologies; memory and programmable logic devices; organization and operation of microcomputers; data/address bus organization; input-output interfacing. Credit will be granted for only one of CPEN 312, CPEN 211. This course is not eligible for Credit/D/Fail grading This course is not eligible for Credit/D/Fail grading. [3-2*-0] Prerequisite: One of ELEC 201, ELEC 204, ELEC 203, MECH 221.',
 'Advanced procedural programming.  Fundamental algorithms for sorting and searching.  Data structures including lists, trees, and hash tables.  Introduction to scripting languages and file input/output. [3-2-0] Prerequisite: APSC 160.',
 'Semiconductor fundamentals; modelling of electronic devices including diodes and transistors; design of power supplies, waveform generators and logic circuits; signals in time and frequency d

In [38]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,CPEN 312,Digital Systems and Microcomputers,Data representation in digital computers; bool...
1,CPSC 259,Data Structures and Algorithms for Electrical ...,Advanced procedural programming. Fundamental ...
2,ELEC 302,Electronic Circuits for Electromechanical Design,Semiconductor fundamentals; modelling of elect...
3,ELEC 343,Electromechanics,Electromechanical energy conversion; linear ac...
4,MECH 306,Data Analysis and Mechatronics Laboratories,The use of probability and statistical methods...
5,MECH 325,Machine Design,"Selection of flexible drives, bearings, fluid ..."
6,MECH 328,Mechanical Engineering Design Project,"Applying the design process, making educated a..."
7,MECH 360,Mechanics of Materials,"Beam deflections, singularity functions; use o..."
8,MECH 366,Modelling of Mechatronic Systems,"Modelling of mechanical, electrical, thermal, ..."
9,MECH 375,Heat Transfer,Steady and transient conduction. Radiation hea...


In [39]:
df.to_csv('UBC_MechEng_Mechatronics_option_core_(Years2-4)_Courses.csv', index = False)

# Thermofluids

## 1. Collect course codes

In [40]:
url = "http://www.calendar.ubc.ca/vancouver/index.cfm?tree=12,195,272,43"
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [41]:
table = page_soup.find("h4", text = "Mechanical Engineering Thermofluids Option").findNext("table")
containers = table.findAll("td")
course_codes = [container.text.strip() for container in containers if bool(re.match("[A-Z]{4} [0-9]{3}", container.text.strip()))]
course_codes

['MECH 305',
 'MECH 325',
 'MECH 327',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 375',
 'MECH 380',
 'MECH 386',
 'APSC 450',
 'ELEC 344',
 'MECH 431',
 'MECH 4542',
 'MECH 463',
 'MECH 466',
 'MECH 479',
 'MECH 489']

In [42]:
course_codes = [course_code[:-1] if len(course_code) > 8 else course_code for course_code in course_codes]
course_codes

['MECH 305',
 'MECH 325',
 'MECH 327',
 'MECH 328',
 'MECH 358',
 'MECH 360',
 'MECH 375',
 'MECH 380',
 'MECH 386',
 'APSC 450',
 'ELEC 344',
 'MECH 431',
 'MECH 454',
 'MECH 463',
 'MECH 466',
 'MECH 479',
 'MECH 489']

In [43]:
len(course_codes)

17

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [44]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "MATH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=MATH",
    "ELEC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=ELEC",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC",
    "CPEN": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=CPEN",
    "CPSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=CPSC"
}

In [45]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  MECH 305
scraped  MECH 325
scraped  MECH 327
scraped  MECH 328
scraped  MECH 358
scraped  MECH 360
scraped  MECH 375
scraped  MECH 380
scraped  MECH 386
scraped  APSC 450
scraped  ELEC 344
scraped  MECH 431
scraped  MECH 454
scraped  MECH 463
scraped  MECH 466
scraped  MECH 479
scraped  MECH 489
17


## 3. Inspect and write to CSV

In [46]:
course_names

['Data Analysis and Mechanical Engineering Laboratories',
 'Machine Design',
 'Thermodynamics II',
 'Mechanical Engineering Design Project',
 'Engineering Analysis',
 'Mechanics of Materials',
 'Heat Transfer',
 'Fluid Dynamics',
 'Industrial Fluid Mechanics',
 'Professional Engineering Practice',
 'Applied Electronics and Electromechanics',
 'Engineering Economics',
 'Thermofluids Capstone Design Project',
 'Mechanical Vibrations',
 'Automatic Control',
 'Introduction to Computational Fluid Dynamics',
 'Experimental Thermofluids']

In [47]:
course_descs

['The use of probability and statistical methods for engineering applications. Mechanics of materials, heat transfer, and fluid dynamics. Instrumentation, data acquisition, and data manipulation using modern computational tools. This course is not eligible for Credit/D/Fail grading. [3-5-1*] Prerequisite: All of MECH 220, MECH 223, MECH 224, MECH 225.',
 'Selection of flexible drives, bearings, fluid power system components, and couplings. Design of bolted joints and power screws. Design and selection of gears, gear trains, and mechanisms. This course is not eligible for Credit/D/Fail grading. [3-0-3] Prerequisite: One of MECH 223, MECH 260. Corequisite: MECH 360.',
 'Air standard and vapor cycles; first and second law of cycles. Exergy. Gas mixtures.  Energy conservation. Equilibrium. Reacting systems. Application to thermofluid systems such as power plants. This course is not eligible for Credit/D/Fail grading. [3-0-1] Prerequisite: Either (a) MECH 225 or (b) one of CHBE 241, ENPH 25

In [48]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,MECH 305,Data Analysis and Mechanical Engineering Labor...,The use of probability and statistical methods...
1,MECH 325,Machine Design,"Selection of flexible drives, bearings, fluid ..."
2,MECH 327,Thermodynamics II,Air standard and vapor cycles; first and secon...
3,MECH 328,Mechanical Engineering Design Project,"Applying the design process, making educated a..."
4,MECH 358,Engineering Analysis,Fourier series; auto- and cross-correlation; p...
5,MECH 360,Mechanics of Materials,"Beam deflections, singularity functions; use o..."
6,MECH 375,Heat Transfer,Steady and transient conduction. Radiation hea...
7,MECH 380,Fluid Dynamics,"Review of principles, compressible flow, poten..."
8,MECH 386,Industrial Fluid Mechanics,"Analysis of piping networks. Review of pumps, ..."
9,APSC 450,Professional Engineering Practice,Legislation affecting the practice of engineer...


In [49]:
df.to_csv('UBC_MechEng_Thermofluids_option_core_(Years2-4)_Courses.csv', index = False)